In [2]:
import pandas as pd
import numpy as np 

In [3]:
# read in the energy matrix
data = pd.read_csv("brewster_matrixS2.txt", sep=" ", comment="#", header=None)
data.columns = ['A','C','G','T']
data.head()

,A,C,G,T
0,-0.313427,-0.198978,-0.291059,-0.237371
1,-0.280792,-0.092443,-0.313427,-0.117770
2,-0.158850,-0.196853,-0.212634,-0.313427
3,-0.313427,-0.179702,-0.312079,-0.184001
4,0.065133,-0.133555,-0.313427,0.495313


In [4]:
seq = "ATGATAGATAGATAGAATTTATATACGCTCGACCCCAGCTA"

In [5]:
# how to get a specific value
data.loc[0,'A']

-0.3134273224466267

In [9]:
def energy(seq1):
    se = 0
    for i in range(len(seq1)):
        if seq[i]=="A":
            e=data.loc[i,'A']
        elif seq[i]=="G":
            e=data.loc[i,'G']
        elif seq[i]=="T":
            e=data.loc[i,'T']
        elif seq[i]== "C":
            e=data.loc[i,'C']
        else:
            e=0
        se+=e
    return(se)

In [29]:
range(len(seq))

range(0, 41)

In [10]:
print(energy(seq))

4.084190419144069
